In [1]:
import os, json, locale, requests, dash, dash_table, dash_auth
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objects as go
from os import listdir
import plotly.figure_factory as ff
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
from datetime import date,datetime,timedelta,time

#--! Check if app is deployed
try:
    with open('./conf/credentials.json') as json_file:
        credentials = json.load(json_file)
    with open('./conf/configuration.json') as json_file:
        config = json.load(json_file)
except:
    raise Exception('Draai eerst deploy.py!')

In [2]:
#--! Set all global variables
globals = {'config': config, 'credentials': credentials, 'styles': {}}
board_url = 'https://api.trello.com/1/members/me/boards?fields=name&key='+credentials.get('API key')+ "&token="+credentials.get('API token')
boards = json.loads(json.dumps(requests.get(board_url).json()))
globals['boards'] = boards
globals['styles']['maindivs'] = {'box-shadow': '8px 8px 8px grey',
                                'background-image': """url('./assets/left.png')""",
                                'background-repeat': 'no-repeat',
                                'background-position': '0px 0px',
                                'margin-top': '1%', 
                                'margin-bottom': '1%', 
                                'margin-left': '1%',
                                'margin-right': '1%',
                                'text-align': 'center',
                                'border-radius': '10px'                   
                                }
globals['styles']['tabs'] = {'border-style': 'solid',
                            'border-width': '2px',
                            'background': 'rgb(255,255,255)',
                            'background': 'radial-gradient(circle, rgba(255,255,255,1) 0%, rgba(162,162,162,1) 100%, rgba(255,255,255,1) 100%)',
                            'margin-top': '5px', 
                            'margin-bottom': '5px', 
                            'margin-right': '5px', 
                            'margin-left': '5px',
                            'border-radius': '6px'
                            }




In [19]:
value = '5dbae5ac432f366ac278036b'

In [88]:
%time
keys = "key="+credentials.get('API key')+"&token="+credentials.get('API token')
trello_base_url = "https://api.trello.com/1/"
board_url = trello_base_url+"boards/"+ value
url_cards = board_url+"?cards=all&card_pluginData=true&card_attachments=true&card_customFieldItems=true&filter=all&"+keys
url_lists = board_url+"/lists?filter=all&"+keys
url_customfields = board_url+"/customFields?"+keys
url_labels = board_url+"/labels?"+keys
url_members = board_url+"/members?"+keys

board = json.loads(json.dumps(requests.get(url_cards).json()))
lists = json.loads(json.dumps(requests.get(url_lists).json()))
customfields = json.loads(json.dumps(requests.get(url_customfields).json()))
labels = json.loads(json.dumps(requests.get(url_labels).json()))
members = json.loads(json.dumps(requests.get(url_members).json()))
cards = board['cards']

def dateCalc(date):
    try:
        newdate = datetime.strptime(date[0:19],'%Y-%m-%dT%H:%M:%S')
        return newdate
    except:
        return None

customfields_dict = {'date': {},'list': {}, 'text': {}, 'number': {}, 'checkbox': {}}
for i in customfields:
    customfields_dict[i['type']] = {}
for i in customfields:
    customfields_dict[i['type']][i['id']] = {}
for i in customfields:
    if i['type'] == 'list':
        customfields_dict[i['type']][i['id']]['name'] = i['name']
        customfields_dict['list'][i['id']]['options'] = {}
        for j in i['options']:
            customfields_dict['list'][i['id']]['options'][j['id']] = j['value'].get('text')
    else:
        customfields_dict[i['type']][i['id']]['name'] = i['name']

chosenlists = []
for i in config.get('Not Started'):
    chosenlists.append(i)
chosenlists.extend(config.get('Blocked'))
chosenlists.extend(config.get('Doing'))
chosenlists.extend(config.get('Done'))
if config['Script options']['Calculate hours'] == True:
    for i in config.get('Epics'):
        chosenlists.append(i)
    for i in config.get('Always continuing'):
        chosenlists.append(i)
    for i in config.get('List with Epics Done'):
        chosenlists.append(i)

def idtodate(cardid):
    hex = cardid[0:8]
    timestamp = int(hex,16)
    timedate = datetime.fromtimestamp(timestamp)
    return timedate    






CPU times: user 15 µs, sys: 1e+03 ns, total: 16 µs
Wall time: 32.2 µs


In [129]:
for i in cards:
    if i['id'] == '5dc0203a9c8166450e95c696':
        for j in i['attachments']:
            print(j['url'])

https://trello-attachments.s3.amazonaws.com/5dbae5ac432f366ac278036b/5dc0203a9c8166450e95c696/b84da870e735233b49895abced9c30d0/image.png
https://trello.com/c/LCabyqHt/242-berichtenverkeer#?sf-tasks&epicId=5e61099a008efe40e8e08149&attachmentId=5e610a179c2d686918410129&dateAssigned=2020-03-05T14:17:59.713Z


In [149]:
def get_epic(url):
    try:
        if 'epicId=' in url:
            start = url.find('epicId=')+7
            end = url.find('&attachmentId=')
            return kaarten.get(url[start:end])['Naam']
        else:
            pass
    except:
        pass

In [ ]:
cards.get(url[])

In [134]:
get_epic('https://trello.com/c/LCabyqHt/242-berichtenverkeer#?sf-tasks&epicId=5e61099a008efe40e8e08149&attachmentId=5e610a179c2d686918410129&dateAssigned=2020-03-05T14:17:59.713Z')





5e61099a008efe40e8e08149


'Berichtenverkeer'

In [187]:
%time
kaarten = {i['id']: {'Naam': i['name'],
                     'KaartID': i['id'],
                     'ListID': i['idList'],
                     'customfields': i['customFieldItems'],
                     'Aangemaakt': idtodate(i['id']),
                     'labels': [label['name'] for label in i['labels'] if i['labels'] != []],
                     'members': [member['fullName'] for member in members if member['id'] in i['idMembers']],
                     'Sjabloon': i['isTemplate'],
                     'Vervaldatum': dateCalc(i['due']),
                     'Gearchiveerd': i['closed'],
                     'Epic': [get_epic(j['url']) for j in i['attachments']],
                     'attachments': {},
                     'shortUrl': i['shortUrl']
                    } for i in cards}
for i,j in kaarten.items():
    while None in j['Epic']:
        j['Epic'].remove(None)
    if j['Epic'] == []:
        j['Epic'] = 'Geen epic'
    else:
        j['Epic'] = j['Epic'][0]


CPU times: user 14 µs, sys: 2 µs, total: 16 µs
Wall time: 31.5 µs


In [175]:
for i,j in kaarten.items():
    while None in j['attachmentsepics']:
        j['attachmentsepics'].remove(None)
#     for k in j['attachmentsepics']:
        
#         if k == None:
#             j['attachmentsepics'].remove(k)
        
#         pass
#     print(j['attachmentsepics'])

In [188]:
for i,j in kaarten.items():
    print(j['Epic'])

Geen epic
Geen epic
Geen epic
Geen epic
Geen epic
Geen epic
Geen epic
Geen epic
Geen epic
Geen epic
Geen epic
Geen epic
Geen epic
Geen epic
Geen epic
Geen epic
Geen epic
Geen epic
Managementinformatie
Digitaliseren
Geen epic
Managementinformatie
Geen epic
Optimalisatie systemen
Geen epic
Geen epic
Geen epic
Optimalisatie werkprocessen
Geen epic
Optimalisatie Bedrijfsbureau
Optimalisatie werkprocessen
Digitaliseren
Optimalisatie werkprocessen
Berichtenverkeer
Managementinformatie
Managementinformatie
Geen epic
Digitaliseren
Managementinformatie
Managementinformatie
Optimalisatie werkprocessen
Optimalisatie werkprocessen
Aanbesteding / inkoop software
Aanbesteding / inkoop software
Dashboards bouwen
Dashboards bouwen
Dashboards bouwen
Dashboards bouwen
Regulier beheer
Aanbesteding 2021
Aanbesteding 2021
Optimalisatie Bedrijfsbureau
Optimalisatie systemen
Optimalisatie werkprocessen
Geen epic
Optimalisatie werkprocessen
Optimalisatie werkprocessen
Optimalisatie werkprocessen
Managementinf

In [48]:
%time


for j in i['attachments']:
    try:
        if 'epicId=' in j['url']:
            start = j['url'].find('epicId=')+7
            end = j['url'].find('&attachmentId=')
            kaarten[i['id']]['epicid'] = j['url'][start:end]
    except:
        pass

CPU times: user 14 µs, sys: 2 µs, total: 16 µs
Wall time: 31.5 µs


In [26]:
kaarten

{'5e61099a008efe40e8e08149': {'Naam': 'Berichtenverkeer'},
 '5e5eb3705364a424f523eeeb': {'Naam': 'SSD v8'},
 '5e5fa029523bd624dc2560c0': {'Naam': 'Managementinformatie'},
 '5e416747ccde752ee087c4d6': {'Naam': 'Optimalisatie werkprocessen'},
 '5dbb19066f233e4f064db0e5': {'Naam': 'Nieuwe huisstijl'},
 '5e5fae50cb6b784a298342de': {'Naam': 'Inburgering'},
 '5e5faec49add94848a4e585f': {'Naam': 'Inlichtingenbureau Inkomen'},
 '5e5faee43ff12c2eaabf78bf': {'Naam': 'Aanbesteding / inkoop software'},
 '5e5fc2bee42fc50c7ddc343a': {'Naam': 'SSD v9'},
 '5e5fc2e5634892428e94ea93': {'Naam': 'Digitaliseren'},
 '5e5fc30d77e1b7495bed0530': {'Naam': 'Optimalisatie Bedrijfsbureau'},
 '5e5fc31cd902483229f85a10': {'Naam': 'Aanbesteding 2021'},
 '5e5fc31eb094232a6e1096b5': {'Naam': 'Ontvlechting Sociaal Team'},
 '5e5fc32c40624337de9bb4f7': {'Naam': "DMO's"},
 '5e5fc376c357d202fb0f88d2': {'Naam': 'Productieverantwoording 2019'},
 '5e610b03955b4a415690505e': {'Naam': 'Optimalisatie systemen'},
 '5e6f387ccbdaab

In [17]:
%time
create_maindiv('5dbae5ac432f366ac278036b')

CPU times: user 13 µs, sys: 2 µs, total: 15 µs
Wall time: 28.4 µs


In [ ]:
customfields_dict = {'date': {i['type']}}